In [ ]:
import os
import torch
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

class PINNs():
    def __init__(self, path):
        self.activation = torch.nn.Tanh()
        model1, model2, model3 = torch.load(path, map_location=torch.device('cpu'))
        self.weights1, self.biases1 = model1
        self.weights2, self.biases2 = model2
        self.weights3, self.biases3 = model3
        self.device = torch.device('cpu')

    def neural_net(self, X, weights, biases):
        H = X
        for i in range(len(weights) - 1):
            w, b = weights[i].float().to(self.device), biases[i].float().to(self.device)
            H = self.activation(torch.mm(H, w) + b)
        w, b = weights[-1].float().to(self.device), biases[-1].float().to(self.device)
        return torch.mm(H, w) + b
    
    def net_u(self, x, t, weights, biases):
        return self.neural_net(torch.cat([x, t], 1), weights, biases)

    def predict(self, x, t):
        x = torch.tensor(x).float().to(self.device)
        t = torch.tensor(t).float().to(self.device)

        u1 = self.net_u(x, t, self.weights1, self.biases1).cpu().detach().numpy()
        u2 = self.net_u(x, t, self.weights2, self.biases2).cpu().detach().numpy()
        u3 = self.net_u(x, t, self.weights3, self.biases3).cpu().detach().numpy()
        return u1, u2, u3

# 模型路径
model_path = 'upload/xpinn_2d_possion_model.pt'
model = PINNs(model_path)

# 加载数据
data = scipy.io.loadmat('upload/XPINN_2D_PoissonEqn.mat')
x_f1 = data['x_f1'].flatten()[:, None]
y_f1 = data['y_f1'].flatten()[:, None]
x_f2 = data['x_f2'].flatten()[:, None]
y_f2 = data['y_f2'].flatten()[:, None]
x_f3 = data['x_f3'].flatten()[:, None]
y_f3 = data['y_f3'].flatten()[:, None]

xi1 = data['xi1'].flatten()[:, None]
yi1 = data['yi1'].flatten()[:, None]
xi2 = data['xi2'].flatten()[:, None]
yi2 = data['yi2'].flatten()[:, None]
xb = data['xb'].flatten()[:, None]
yb = data['yb'].flatten()[:, None]
u_exact = data['u_exact'].flatten()[:, None]

# 构建输入数据
X_star1 = np.hstack((x_f1.flatten()[:, None], y_f1.flatten()[:, None]))
X_star2 = np.hstack((x_f2.flatten()[:, None], y_f2.flatten()[:, None]))
X_star3 = np.hstack((x_f3.flatten()[:, None], y_f3.flatten()[:, None]))

# 预测解
u_pred1, _, _ = model.predict(X_star1[:, 0:1], X_star1[:, 1:2])
_, u_pred2, _ = model.predict(X_star2[:, 0:1], X_star2[:, 1:2])
_, _, u_pred3 = model.predict(X_star3[:, 0:1], X_star3[:, 1:2])

# 用于绘图
X1, Y1 = X_star1[:, 0:1], X_star1[:, 1:2]
X2, Y2 = X_star2[:, 0:1], X_star2[:, 1:2]
X3, Y3 = X_star3[:, 0:1], X_star3[:, 1:2]

# 拼接子域的解
u_pred = np.concatenate([u_pred1, u_pred2, u_pred3])

# 计算误差
error_u_total = np.linalg.norm(np.squeeze(u_exact) - u_pred.flatten(), 2) / np.linalg.norm(np.squeeze(u_exact), 2)
print('Error u_total: %e' % (error_u_total))

# 分割真实解
u1 = u_exact[:u_pred1.shape[0]]
u2 = u_exact[u_pred1.shape[0]: u_pred1.shape[0] + u_pred2.shape[0]]
u3 = u_exact[u_pred1.shape[0] + u_pred2.shape[0]: u_pred1.shape[0] + u_pred2.shape[0] + u_pred3.shape[0]]

# 绘图
fig = plt.figure(figsize=(20, 5))
norm = plt.Normalize(vmin=0, vmax=8)

plt.subplot(131)
plt.scatter(X1, Y1, c=u_pred1, cmap='rainbow', norm=norm)
plt.scatter(X2, Y2, c=u_pred2, cmap='rainbow', norm=norm)
plt.scatter(X3, Y3, c=u_pred3, cmap='rainbow', norm=norm)
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()

plt.subplot(132)
plt.scatter(X1, Y1, c=u1, cmap='rainbow', norm=norm)
plt.scatter(X2, Y2, c=u2, cmap='rainbow', norm=norm)
plt.scatter(X3, Y3, c=u3, cmap='rainbow', norm=norm)
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()

plt.subplot(133)
plt.scatter(X1, Y1, c=np.abs(u_pred1 - u1), cmap='rainbow')
plt.scatter(X2, Y2, c=np.abs(u_pred2 - u2), cmap='rainbow')
plt.scatter(X3, Y3, c=np.abs(u_pred3 - u3), cmap='rainbow')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()

plt.suptitle("2D Poisson Error: %.3e" % (error_u_total), fontsize=20)
plt.show()
